## CS2101 - Programming for Science and Finance
Prof. Götz Pfeiffer<br />
School of Mathematical and Statistical Sciences<br />
University of Galway

***

### Objects and Classes
# Week 4: Inheritance, Matrices

## Inheritance

In [ ]:
class Person:
    def __init__(self, first, last):
        self.firstname = first
        self.lastname = last
    def __repr__(self):
        return f"{self.firstname} {self.lastname}"
    def chat(self):
        print(f"Hi, I'm {self}. How are you getting on?")

In [ ]:
john = Person("John", "Kelly")
john

In [ ]:
john.chat()

* A `Student` is like a `Person`, except that it **has** a student id, and that it **use** that to log in.

In [ ]:
class Student:
    def __init__(self, first, last, number):
        self.firstname = first
        self.lastname = last
        self.student_id = number
    def __repr__(self):
        return f"{self.firstname} {self.lastname}"
    def chat(self):
        print(f"Hi, I'm {self}. How are you getting on?")
    def login(self):
        print(f"login: {self.student_id}")

In [ ]:
anna = Student("Anna", "Byrne", 4321)
anna.chat()

In [ ]:
anna.login()

* Inheritance allows us to avoid this kind of repetition.
* We can define the `Student` class as a **subclass** of `Person`, to express the fact that a student is a special kind of person.
* Then a `Student` object has all the **attributes** of a `Person` object, and it can avail of all the **methods** defined in the `Person` class.

In [ ]:
class Student(Person):
    pass

In [ ]:
anna = Student("Anna", "Byrne")
anna.chat()

* A `Student` can have additional attributes.
* The `Student` class can define additional methods, and also override existing methods.

In [ ]:
class Student(Person):
    def __init__(self, first, last, number):
        self.firstname = first
        self.lastname = last
        self.student_id = number
    def login(self):
        print(f"login: {self.student_id}")

In [ ]:
anna = Student("Anna", "Byrne", 4321)
anna

In [ ]:
anna.chat()

In [ ]:
anna.login()

* Using the `super` function,  we can delegate parts of the initializtion to the `Student`'s **superclass** `Person`.

In [ ]:
class Student(Person):
    def __init__(self, first, last, number):
        super().__init__(first, last)
        self.student_id = number
    def login(self):
        print(f"login: {self.student_id}")

In [ ]:
anna = Student("Anna", "Byrne", 4321)
anna

In [ ]:
anna.chat()
anna.login()

* A `Teacher` is another special kind of `Person`, except that it has a `title` attribute which is used in their salutation.
* So: inherit from `Person`, redefine `__init__`, add `title` attribute, redefine `hello`.

In [ ]:
class Teacher(Person):
    def __init__(self, first, last, title):
        super().__init__(first, last)
        self.title = title
    def __repr__(self):
        return f"{self.title} {self.lastname}"
    def chat(self):
        print(f"Hi, I'm {self}. Have you done your homework?")

* Make a `Teacher` object and let them talk ...

In [ ]:
teacher = Teacher("Steven", "Kennedy", "Mr.")
teacher.chat()

* In case you were wondering, there is a way to make a `Teacher` behave like a normal `Person` ...

In [ ]:
super(Teacher, teacher).chat()

##  Matrix Algebra

* Recall the `Vector` class (with some additional special methods):

In [ ]:
class Vector:
    def __init__(self, *data):
        self.data = data
    def __repr__(self):
        return f"Vector{self.data}"
    def __len__(self):
        return len(self.data)
    def __getitem__(self, i):
        return self.data[i]
    def __eq__(self, other):
        return all(x == y for x, y in zip(self, other))
    def __add__(self, other):
        return Vector(*[x + y for x, y in zip(self, other)])
    def __rmul__(self, other):
        return Vector(*[other * x for x in self])
    def __neg__(self):
        return -1 * self
    def __sub__(self, other):
        return self + -other
    def __and__(self, other):
        return sum(x * y for x, y in zip(self, other))

* With this class, we can compute for example the following: 

In [ ]:
v = Vector(1,2,3)
w = Vector(4,5,6)
print(v + w)
print(w - v)
print(0*v)
print(0*v == 0*w)

* Here we have in particular implemented one more special method `__and__` to allow expressions of the form `v & w` for `Vector` objects `v` and `w`,
* The meaning of `v & w` shall be the inner product of the vectors `v` and `w`.
* Recall (from last week's exercises) that the **inner product** of two vectors $(a_1, a_2, \dots, a_n)$ and $(b_1, b_2, \dots, b_n)$ is the number
 $$
a_1 b_1 + a_2 b_2 + \dots + a_n b_n
 $$
* We will use this inner product to simplify the implementation of matrix multiplication which comes next, after defining a suitable `Matrix` class.

In [ ]:
print(v & w)

* A **matrix** can be regarded as a list of vectors, the **rows** of the matrix.
* That is, we want to represent the matrix
  $$
  \left[\begin{array}{ccc}
  1&2&3\\4&5&6
  \end{array}\right]
  $$
  as python object
  ```python
  Matrix(Vector(1,2,3), Vector(4,5,6))
  ```
* So we need a new class `Matrix`.

In [ ]:
class Matrix:
    def __init__(self, *data):
        self.data = data
    def __repr__(self):
        return f"Matrix{self.data}"

In [ ]:
m = Matrix(v, w)
m

* Matrices are like vectors, they can be **added** and **scaled**.
* **Delegation**: keeping in mind that a matrix is a list of vectors, and that `Vector` objects already know how to add and scale, we can keep the corresponding methods for `Matrix` objects short and simple.

In [ ]:
class Matrix:
    def __init__(self, *data):
        self.data = data
    def __repr__(self):
        return f"Matrix{self.data}"
    def __len__(self):
        return len(self)
    def __getitem__(self, i):
        return self.data[i]
    def __add__(self, other):
        return Matrix(*[x + y for x, y in zip(self, other)])
    def __rmul__(self, other):
        return Matrix(*[other * x for x in self])

In [ ]:
m = Matrix(v, w)
m + m

In [ ]:
3 * m

*  Now, there is in fact a lot of repetition between the `Matrix` and the `Vector` class.
*  Perhaps a `Matrix` is just a special kind of `Vector`, one whose entries are `Vectors` rather than numbers?
*  Let's try and use inheritance to reflect this relationship.
*  Since `__init__`, `__len__` and `__getitem__` are literally the same code in both classes, we can omit them from the `Matrix` class and instead **inherit** them from `Vector`.

In [ ]:
class Matrix(Vector):
    def __repr__(self):
        return f"Matrix{self.data}"
    def __add__(self, other):
        return Matrix(*[x + y for x, y in zip(self, other)])
    def __rmul__(self, other):
        return Matrix(*[other * x for x in self])

In [ ]:
m = Matrix(v, w)
m + m

* In fact, `__eq__`, `__neg__` and `__sub__` will also work as they should with matrices

In [ ]:
m + m + m == 3 * m

In [ ]:
m - m == 0*m

* Note that even the $3$ methods defined in the `Matrix` class look very much like their `Vector` counterparts.
* The only difference is that one returns (or prints) a `Vector` object, where the other returns a `Matrix` object.
* Wouldn't it be nice if we could derive this **type** information from the object and use it programmatically ...
* **Introspection**: the vector `v` knows that it is a `Vector` object, the matrix `m` knows that it is a `Matrix` object.
* Each python object knows its `type`.
* And types are objects ...  that have names ...

In [ ]:
type(v), type(m)

* Since the type is a class, it can be used to create objects.

In [ ]:
type(v)(1,1,1)

* And the **name** of a type (or class) is contained in its `__name__` component.

In [ ]:
type(v).__name__

* Using this, we can remove the explict references to the class name from the `Vector` class as follows.

* We replace `__repr__` with
  ```python
    def __repr__(self):
        return f"{type(self).__name__}{self.data}" 
  ```

* We replace `__add__` with
  ```python
    def __add__(self, other):
        return type(self)(*[x + y for x, y in zip(self, other)])
  ```

* And we replace `__rmul__` with
  ```python
    def __rmul__(self, other):
        return type(self)(*[other * x for x in self])
  ```

* Here it goes:

In [ ]:
class Vector:
    def __init__(self, *data):
        self.data = data
    def __repr__(self):
        return f"{type(self).__name__}{self.data}"
    def __len__(self):
        return len(self.data)
    def __getitem__(self, i):
        return self.data[i]
    def __eq__(self, other):
        return all(x == y for x, y in zip(self, other))
    def __add__(self, other):
        return type(self)(*[x + y for x, y in zip(self, other)])
    def __rmul__(self, other):
        return type(self)(*[other * x for x in self])
    def __neg__(self):
        return -1 * self
    def __sub__(self, other):
        return self + -other
    def __and__(self, other):
        return sum(x * y for x, y in zip(self, other))

* Test drive, again ...

In [ ]:
v = Vector(1,2,3)
w = Vector(4,5,6)
v, w

In [ ]:
w - v

In [ ]:
3 * w == w + w + w

* And then:  a `Matrix` is a special kind of `Vector` ...

In [ ]:
class Matrix(Vector):
    pass

In [ ]:
m = Matrix(v, w)
m

In [ ]:
m + m

In [ ]:
3 * m

In [ ]:
3 * m == m + m + m

In [ ]:
m - m == 0*m

## Matrix Multiplication

* By definition, if $A = (a_{ij})$ and $B = (b_{jk})$ then $A B = C = (c_{ik})$, where
  $$
  c_{ik} = \sum_j a_{ij} b_{jk}
  $$
* On closer inspection the $i,k$-entry of the product matrix $C$ is the **inner product** of the $i$-th **row** of $A$, i.e., the vector
  $$
(a_{i1}, a_{i2}, \dots, a_{in})
  $$
  and the $k$-th **column** of $B$, i.e., the vector
  $$
  (b_{1k}, b_{2k}, \dots, b_{nk})
  $$
* So perhaps we can use our `Vector` inner product `v & w` to compute products of matrices ...

* **Example**.
  $$
  A = \left[
  \begin{array}{ccc}
  1 & 0 & 1 \\
  2 & 1 & 1 \\
  0 & 1 & 1 \\
  1 & 1 & 2
  \end{array}
  \right]
  \qquad
    B = \left[
  \begin{array}{ccc}
  1 & 2 & 1 \\
  2 & 3 & 1 \\
  4 & 2 & 2 \\
  \end{array}
  \right]
  \qquad
  AB = \left[
  \begin{array}{ccc}
  5 & 4 & 3 \\
  8 & 9 & 5 \\
  6 & 5 & 3 \\
  11 & 9 & 6
  \end{array}
  \right]
  $$

* But first, in order to be able to access the **columns** of a matrix (as `Vector`s) we need a way to **transpose** a matrix.
* Recall `zip` ...

In [ ]:
ma = Matrix(
    Vector(1, 0, 1),
    Vector(2, 1, 1),
    Vector(0, 1, 1),
    Vector(1, 1, 2)
)
ma

In [ ]:
list(zip(*ma.data))

In [ ]:
list(zip(*ma))

In [ ]:
[Vector(*x) for x in zip(*ma)]

In [ ]:
Matrix(*[Vector(*x) for x in zip(*ma)])

* We can use this as implementation of a `transpose` method for our `Matrix` class.

In [ ]:
class Matrix(Vector):
    def transpose(self):
        return Matrix(*[Vector(*x) for x in zip(*self)])

In [ ]:
ma = Matrix(
    Vector(1, 0, 1),
    Vector(2, 1, 1),
    Vector(0, 1, 1),
    Vector(1, 1, 2)
)
mb = Matrix(
    Vector(1, 2, 1),
    Vector(2, 3, 1),
    Vector(4, 2, 2)
)

In [ ]:
mb.transpose()

In [ ]:
[v & w for v in ma for w in mb.transpose()]

In [ ]:
[[v & w for v in ma] for w in mb.transpose()]

In [ ]:
[[v & w for w in mb.transpose()] for v in ma]

In [ ]:
Matrix(*[Vector(*[v & w for w in mb.transpose()]) for v in ma])

* But, note how the $k$th column of the product $AB$ is the vector consisting of the inner products of **all** the rows of $A$ with the $k$th column of $B$ ...
* So if we define `m & v` to be the vector of all inner products of the rows of a matrix `m` with a vector `v` ...
* ... then the **matrix product** `ma @ mb` could be computed as transpose of `ma & v` for `v` in `mb.transpose` ...

In [ ]:
class Matrix(Vector):
    def transpose(self):
        return Matrix(*[Vector(*x) for x in zip(*self)])
    def __and__(self, other):
        return Vector(*[x & other for x in self])
    def __matmul__(self, other):
        return Matrix(*[self &  x for x in other.transpose()]).transpose()

In [ ]:
m = Matrix(v, w)
m

In [ ]:
m.transpose()

In [ ]:
m @ m.transpose()

In [ ]:
m.transpose() @ m

In [ ]:
ma = Matrix(
    Vector(1, 0, 1),
    Vector(2, 1, 1),
    Vector(0, 1, 1),
    Vector(1, 1, 2)
)
mb = Matrix(
    Vector(1, 2, 1),
    Vector(2, 3, 1),
    Vector(4, 2, 2)
)

In [ ]:
ma @ mb

## Summary

* Classes often form a hierarchy of related types.
* In python, a class can be defined as a subclass of another class, thereby **inheriting** all of that classes methods.
* Under suitable circumstances, inheritance can save a lot of code duplication and thus structure and simplify a large program.
* Careful use of special methods allows for short implementations of complex tasks.

## References

### Python

* inheritance tutorial
* the `super` function

##  Exercises

* Use the vectors $$u = (6, 8, -1, 0, -7), \qquad v = (-8, 8, 6, 2, 3), \qquad w = (-2, 9, -3, -9, -1)$$ for the next few exercises.

* Represent $u$, $v$ and $w$ as `Vector` objects and compute
  * $1000v$,
  * $u + v + w$,
  * $2u - 3v + 4w$.

In [ ]:
u = Vector(6, 8, -1, 0, 7)
v = Vector(-8, 8, 6, 2, 3)
w = Vector(-2, 9, -3, -9, -1)
print(1000 * v)
print(u + v + w)
print(2*u - 3*v + 4*w)

* Compute the $9$ inner products of all of $u$, $v$ and  $w$ with all of $u$, $v$ and $w$.
* Is the inner product of vectors $x$ and $y$ equal to the inner product of $y$ and $x$? 

In [ ]:
for x in [u, v, w]:
    for y in [u, v, w]:
        print(x & y, end=" ")
    print()

* Let $A$ be the matrix whose **rows** are $u$, $v$, $w$ (in that order) and let $B$ be the matrix whose columns are $u$, $v$, $w$ (in that order).  Represent $A$ and $B$ as `Matrix` objects and compute the matrix products
  * $AB$
  * $BA$
* Is $AB = BA$?
* Does any of thses products remind you of the inner products above?

In [ ]:
A = Matrix(u, v, w)
print(A)
B = A.transpose()
print(B)
print(A @ B)
print(B @ A)

* Compute the product of the matrix `A` and the vector `u`.  What is the relation of the resulting `Vector` to the matrix product $AB$?

In [ ]:
A & u

* Define a function `standardBasisVector` with two arguments, `n` and `i`, which computes and returns the $i$th **standard basis vector** of length $n$ as a `Vector` object, that is a vector with $n$ components, all equal to $0$, except for the $i$ one, which should be $1$.
  So `standardBasisVector(5, 3)` should return `Vector(0,0,0,1,0)`.

In [ ]:
def standardBasisVector(n, i):
    return Vector(*[1 if j==i else 0 for j in range(n)])

In [ ]:
standardBasisVector(5, 3)

* Compute some inner products of standard basis vectors of the same length, and verify that such an inner product is $0$, unless the two basis vectors are the same. 

In [ ]:
for i in range(5):
    for j in range(5):
        print(standardBasisVector(5, i) & standardBasisVector(5, j), end=" ")
    print()


* Define a function `identityMatrix` with one argument `n`, which computes and returns the $n \times n$ identity matrix as a `Matrix` object, that is the matrix with $3$ rows and $3$ columns, with all entries $0$, except for the diagonal, where the entries are $1$. So `identityMatrix(3)` should return `Matrix(Vector(1,0,0), Vector(0,1,0), Vector(0,0,1))`.

In [ ]:
def identityMatrix(n):
    return Matrix(*[standardBasisVector(n, i) for i in range(n)])

In [ ]:
identityMatrix(3)

* Verify that your identity matrices behave like a multiplicative identity: if `I3` is the $3 \times 3$ identity matrix then the product of `I3` and `A` above should be equal to `A`.  Similarly, the product of `B` above and `I3` should be `B`.  What should be true for `I4`, the $4 \times 4$ identity matrix?

In [ ]:
print(identityMatrix(3) @ A == A)
print(B @ identityMatrix(3) == B)

In [ ]:
identityMatrix(4) @ B == B

* Is there a way to get rid of one of the two `transpose` calls in our implementation of matrix multiplication?